Python immmovable property valuation tutorial. 

Making use of modules such as Beautifulsoup and numpy are helpful in valuation. 
Beautifulsoup is particulalrly strong in data collection for comparable sales. Whilst numpy is capable of 
various mathmatical operations including TVM, DCF and other equations so demonstrated in this tutorial. 

In [19]:
import requests
from bs4 import BeautifulSoup
import urls
import csv

pricelist = []
titlelist = []
locationlist = []
addresslist = []
sizelist = []



for x in range(18):
    p24 = f"https://www.property24.com/houses-for-sale/midrand/gauteng/16/p{x}"
    req = requests.get(p24)
    soup = BeautifulSoup(req.text, "html.parser")
    for price in soup.find_all("span", attrs=["p24_price"]):
        pricelist.append(price.text)
    for size in soup.find_all("span", attrs=["p24_size"]):
        sizelist.append(size.text)



pricelist = [price.strip().replace("\n", "").replace("\r", "").replace(" ", "").replace("R", "") for price in pricelist]    
print(f"PRICES: {len(pricelist), pricelist}")

sizelist = [size.strip().replace("\n", "").replace("m²", "").replace(" ", "").replace("ha", "000").replace(".", "") for size in sizelist]
print(f"SIZE LIST FORMATTED: {len(sizelist)} {sizelist}")



PRICES: (363, ['3800000', '850000', '4400000', '1260000', '1180000', '3650000', '2595000', '2600000', '2029075', '2899000', '963000', '4799000', '1750000', '2595000', '3800000', '850000', '4400000', '1260000', '1180000', '3650000', '2595000', '2600000', '2029075', '2899000', '963000', '4799000', '1750000', '3180000', '2500000', '2620000', '2300000', '10500000', '11980000', '5200000', '1749000', '3399000', '1600000', '9499999', '1900000', '11990000', '13499000', '15999000', '3240000', '8499000', '2620000', '1990000', '7999000', '2499000', '1850000', '780000', '14000000', '5495000', '3549000', '2500000', '2499000', '10995000', '8900000', '5600000', '2299000', '3800000', '2450000', '4850000', '2499000', '2800000', '1899000', '2900000', '5500000', '6499000', '2200000', '9999000', '870000', '5400000', '2500000', '6699000', '1350000', '3999000', '6499000', '12999000', '9999000', '670000', '5980000', '15999000', '2049000', '2750000', '11600000', '4499000', '2899000', '7990000', '1900000', '26

Before parsing the data we need to set up variables to hold the data pulled from the website, today we will pull the price, title, location, address and property size from various different urls. 

We use the find_all() parameter to collect all data partaining to the selected parameters, for more information see the Beautifulsoup documentation online.
As seen the data is dirty and requires cleaning. 

The pricelist and sizelist require formatting. 

The lists are converted into numpy arrays to be used with numpy. 
We can now start with the mathmatical aspects of valuation. 
It is clear to first set out what metrics we will need to look at, average house price and house area are helpful in the calculation of land value per m^2. 

In [20]:
import numpy as np

pricelist.pop(0)
pricelist.pop(0)
pricelist.pop(0)
pricelist.pop(0)

nsize = np.array(sizelist, dtype=int)
nprice = np.array(pricelist, dtype=int)
weighted_mean = np.average(nprice, weights=(nsize))
stdsize = np.std(nsize)
stdprice = np.std(nprice)
stdave = stdprice/stdsize




print(f"weighted_mean: {weighted_mean}, stdprice: {stdprice}, stdsize: {stdsize}, other: {stdave}")


weighted_mean: 3822094.5559441, stdprice: 5012160.718480242, stdsize: 24332.099715267806, other: 205.98965059045983


In [23]:
totalprice = nprice.astype(float).sum()
totalsize = nsize.astype(float).sum()

aveprice = totalprice/len(nprice)
avesize = totalsize/len(nsize)
avelandvalue = aveprice/avesize
mediansize = np.median(nsize)
medianprice = np.median(nprice)
medianaveraged = medianprice/mediansize 

print(f"Average Price weighted: {aveprice}, Average area weighted: {avesize}")
print(f"Average land value: {avelandvalue}")
print(f"Average land value weighted: {weighted_mean}")
print(f"Total Price: {totalprice}, Total area: {totalsize}")
print(f"Median price prediction: {medianprice}")
print(f"Median size prediction: {mediansize}")
print(f"Median averaged: {medianaveraged}")

Average Price weighted: 4642888.027855153, Average area weighted: 3775.7298050139275
Average land value: 1229.6663870623622
Average land value weighted: 3822094.5559441
Total Price: 1666796802.0, Total area: 1355487.0
Median price prediction: 2800000.0
Median size prediction: 600.0
Median averaged: 4666.666666666667


The above calculation is used to determine average land value per m^2. This is helpful for establishing a potential price of a valuation if using the GBA. 
The downside to this is certain larger properties also greatly decrease the rand value per m2, while it is true the bigger the property the greater the value, that value is only mainly derived from the physical improvements to the land. In the case of the above data an outlier of 10000m or 1 hectare greatly affects the average land size and price. While other parsed data may include the total area of the land rather than just the GBA. 

In [27]:
with open("prop_data_large.csv", "w") as r:
    writer = csv.writer(r)
    writer.writerow(pricelist)

FAR = 0.5
valuationarea = 232.56
valuationprice = valuationarea * (medianaveraged/FAR)
print(f"valuationprice: {valuationprice, valuationarea, (medianaveraged/FAR)}")

valuationprice: (2170560.0, 232.56, 9333.333333333334)


Above we see the subject property is worth R2 674 603.45
This is fairly close to the stated market value of R 3 000 000.00
An estimate for the house price is within R 2 675 000.00 - R 3 000 000.00